In [54]:
import numpy as np
import pandas as pd
import geopandas as gpd
import warnings
from shapely.geometry import Polygon
import sys
import os
import osmnx as ox
city_geo = ox.geocoder.geocode_to_gdf

In [55]:
thresholds = [300, 600, 1000]

In [56]:
ls_in = ['Abidjan','Addis Ababa','Baghdad','Bangalore','Belgrade','Belo Horizonte','Bogota','Bologna','Cali','Canberra',\
         'Casablanca','Chandigarh','Chelyabinsk','Chengdu','Christchurch','Cochabamba','Copenhagen','Curitiba','Damascus',\
         'Detroit','Dhaka','Dongguan','Fortaleza','Ghent','Guatemala City','Gwangju','Hanoi','Indore','Johannesburg',\
         'Kampala','Karlsruhe','Kathmandu','Kharkiv','Kuala Lumpur','Kumamoto','Lagos','Leicester','Liverpool',\
         'Lubumbashi','Lviv','Manila','Memphis','Mexico City','Mombasa','Nanjing','New Delhi','Oran','Oslo','Perth',\
         'Philadelphia','Pune','Rennes','Rosario','Rostov','San Antonio','San Diego','Santo Domingo','Seattle','Seoul',\
         'Sevilla','Shijiazhuang','Shiraz','Skopje','Surabaya','Tallinn','Tashkent','Tel Aviv','Toronto','Turin',\
         'Utrecht','Utsunomiya','Washington DC','Yaounde','Yerevan','Zibo']

In [ ]:
ls_in = ['Toronto']

In [13]:
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(ls_in)]
cities_adj = cities_adj.sort_values('City').reset_index()

ls_adj = ['Canberra','Karlsruhe','Oslo']

cities_adj2 = cities[cities['City'].isin(ls_adj)]
cities_adj2 = cities_adj2.sort_values('City').reset_index()

In [56]:
warnings.filterwarnings('ignore')
ser = gpd.GeoSeries(crs = 4326)
for i in cities_adj['OSM_area']:
    ser = pd.concat([ser, city_geo(i.split(', ')).dissolve().geometry.centroid])
ser.index = ls_in
ser

Oslo    POINT (10.73849 59.97554)
dtype: geometry

In [57]:
ser.to_file('D:/Dumps/GEE-WP Scores/cities_2.gpkg')

In [58]:
path = "D:/Dumps/GEE-WP Scores/Gravity/100m grids/Grid_geoms/"
ls_file = os.listdir(path)
ls_file = np.array(ls_file)[[i.rsplit('_')[1] == 'grav3' for i in ls_file]]
ls_file = ls_file[[any([j in i for j in ls_in]) for i in ls_file]]

ls_2 = ls_file[[any([j in i for j in ls_adj]) for i in ls_file]]

warnings.filterwarnings('ignore')
area = [city_geo(i.split(', ')).dissolve().geometry for i in cities_adj2['OSM_ineq_adj']]
files = [gpd.read_file(path+i) for i in ls_2]
scores = [files[i].overlay(gpd.GeoDataFrame(geometry = area[i], crs = 4326)) for i in range(len(files))]

In [59]:
[i[1].to_file('D:/Dumps/GEE-WP Scores/Gravity/100m grids/grid_geoms_adj/gridscore_grav3_'+cities_adj2['City'][i[0]]+'.gpkg')\
 for i in enumerate(scores)]

In [57]:
%%time
warnings.filterwarnings('ignore')

path = "D:/Dumps/GEE-WP Scores/Gravity/100m grids/Grid_geoms_adj/"
ls_file = os.listdir(path)
ls_file = np.array(ls_file)[[i.rsplit('_')[1] == 'grav3' for i in ls_file]]
ls_file = ls_file[[any([j in i for j in ls_in]) for i in ls_file]]

#ls_2 = ls_file[[any([j in i for j in ls_adj]) for i in ls_file]]

#warnings.filterwarnings('ignore')
#area = [city_geo(i.split(', ')).dissolve().geometry for i in cities_adj2['OSM_ineq_adj']]
#files = [gpd.read_file(path+i) for i in ls_2]
#scores = [files[i].overlay(gpd.GeoDataFrame(geometry = area[i], crs = 4326)) for i in range(len(files))]

print('preorocessing done')
print('')

cities_scores = list()
for i in enumerate(ls_file):
    city_score = gpd.read_file(path+i[1])
    
    #k = 0
    #if i[1] in ls_adj:
        #city_score2 = scores[k]
        #k = k+1
    #else:
    #    city_score2 = city_score
    
    scores_df = city_score.iloc[range(0,len(city_score),3)]['geometry'].reset_index(drop = True)
    for j in thresholds:
        ls = list()
        score = city_score['tr_'+str(j)].dropna().reset_index(drop =True)
        for k in score:
            if k == 0: 
                ls.append('0 no')
            elif k < j: # one perfect score (no route cost to UGS); score is threshold - route (and entry) cost
                ls.append('1 low')
            elif k < j*2: # two perfect scores
                ls.append('2 mediocre')
            elif k < j*3: # three perfect scores
                ls.append('3 sufficient')
            elif k < j*4: # four perfect scores
                ls.append('4 good')
            else: # above four perfect scores (or 8 UGS on half the threshold distance)
                ls.append('5 excellent')
        print(i[1].rsplit('_')[2].rsplit('.')[0], j)
        scores_df = pd.concat([scores_df, 
                               pd.Series(round(score, 1), name = 'Grav-Sc '+str(j)), 
                               pd.Series(ls, name = 'GravQnt '+str(j))], 
                              axis = 1)
    cities_scores.append(scores_df)
cities_scores[0]

preorocessing done

Abidjan 300
Abidjan 600
Abidjan 1000
Addis Ababa 300
Addis Ababa 600
Addis Ababa 1000
Baghdad 300
Baghdad 600
Baghdad 1000
Bangalore 300
Bangalore 600
Bangalore 1000
Belgrade 300
Belgrade 600
Belgrade 1000
Belo Horizonte 300
Belo Horizonte 600
Belo Horizonte 1000
Bogota 300
Bogota 600
Bogota 1000
Bologna 300
Bologna 600
Bologna 1000
Cali 300
Cali 600
Cali 1000
Canberra 300
Canberra 600
Canberra 1000
Casablanca 300
Casablanca 600
Casablanca 1000
Chandigarh 300
Chandigarh 600
Chandigarh 1000
Chelyabinsk 300
Chelyabinsk 600
Chelyabinsk 1000
Chengdu 300
Chengdu 600
Chengdu 1000
Christchurch 300
Christchurch 600
Christchurch 1000
Cochabamba 300
Cochabamba 600
Cochabamba 1000
Copenhagen 300
Copenhagen 600
Copenhagen 1000
Curitiba 300
Curitiba 600
Curitiba 1000
Damascus 300
Damascus 600
Damascus 1000
Detroit 300
Detroit 600
Detroit 1000
Dhaka 300
Dhaka 600
Dhaka 1000
Dongguan 300
Dongguan 600
Dongguan 1000
Fortaleza 300
Fortaleza 600
Fortaleza 1000
Ghent 300
Ghent 600
Ghen

,geometry,Grav-Sc 300,GravQnt 300,Grav-Sc 600,GravQnt 600,Grav-Sc 1000,GravQnt 1000
0,"POLYGON ((-4.00208 5.47625, -4.00125 5.47625, ...",0.0,0 no,0.0,0 no,0.0,0 no
1,"POLYGON ((-4.00125 5.47625, -4.00042 5.47625, ...",0.0,0 no,0.0,0 no,0.0,0 no
2,"POLYGON ((-4.00042 5.47625, -3.99958 5.47625, ...",0.0,0 no,0.0,0 no,0.0,0 no
3,"POLYGON ((-4.05042 5.46875, -4.04958 5.46875, ...",0.0,0 no,0.0,0 no,0.0,0 no
4,"POLYGON ((-4.04958 5.46875, -4.04875 5.46875, ...",0.0,0 no,0.0,0 no,0.0,0 no
...,...,...,...,...,...,...,...
62388,"POLYGON ((-4.09292 5.39458, -4.09208 5.39458, ...",0.0,0 no,83.5,1 low,606.9,1 low
62389,"POLYGON ((-4.09208 5.39458, -4.09125 5.39458, ...",0.0,0 no,36.7,1 low,515.0,1 low
62390,"POLYGON ((-4.09125 5.39458, -4.09042 5.39458, ...",0.0,0 no,0.0,0 no,606.0,1 low
62391,"POLYGON ((-4.09042 5.39458, -4.08958 5.39458, ...",0.0,0 no,0.0,0 no,548.0,1 low


In [61]:
path = "D:/Dumps/GEE-WP Scores/E2SFCA/100m grids/Grid_geoms/"
ls_file = os.listdir(path)
ls_file = np.array(ls_file)[[i.rsplit('.')[-1] == 'gpkg' for i in ls_file]]
ls_file = ls_file[[any([j in i for j in ls_in]) for i in ls_file]]

ls_2 = ls_file[[any([j in i for j in ls_adj]) for i in ls_file]]

warnings.filterwarnings('ignore')
area = [city_geo(i.split(', ')).dissolve().geometry for i in cities_adj2['OSM_ineq_adj']]
files = [gpd.read_file(path+i) for i in ls_2]
scores = [files[i].overlay(gpd.GeoDataFrame(geometry = area[i], crs = 4326)) for i in range(len(files))]

In [62]:
[i[1].to_file('D:/Dumps/GEE-WP Scores/E2SFCA/100m grids/grid_geoms_adj/'+cities_adj2['City'][i[0]]+'.gpkg')\
 for i in enumerate(scores)]

In [58]:
%%time
warnings.filterwarnings('ignore')

path = "D:/Dumps/GEE-WP Scores/E2SFCA/100m grids/Grid_geoms_adj/"
ls_file = os.listdir(path)
ls_file = np.array(ls_file)[[i.rsplit('.')[1] == 'gpkg' for i in ls_file]]
ls_file = ls_file[[any([j in i for j in ls_in]) for i in ls_file]]

#ls_2 = ls_file[[any([j in i for j in ls_adj]) for i in ls_file]]

#warnings.filterwarnings('ignore')
#area = [city_geo(i.split(', ')).dissolve().geometry for i in cities_adj2['OSM_ineq_adj']]
#files = [gpd.read_file(path+i) for i in ls_2]
#scores = [files[i].overlay(gpd.GeoDataFrame(geometry = area[i], crs = 4326)) for i in range(len(files))]

print('preorocessing done')
print('')

E2SFCA_scores = list()
for i in enumerate(ls_file):
    city_score = gpd.read_file(path+i[1])
    
    #k = 0
    #if i[1] in ls_adj:
        #city_score2 = scores[k]
        #k = k+1
    #else:
    #    city_score2 = city_score
    
    score_df = city_score[['population','geometry']]
    for j in thresholds:
        ls = list()
        score = city_score['Sc-norm '+str(j)].dropna().reset_index(drop =True)
        for k in score:
            if k == 0: # m2 of UGS per person and weighted with Gaussian (threshold capped) distance decay
                ls.append('0 no')
            elif k < 9: # m2 per person required by the WHO
                ls.append('1 low')
            elif k < 18: # m2 per person twice required by the WHO
                ls.append('2 mediocre')
            elif k < 30: # m2 per person preferred by the WHO
                ls.append('3 sufficient')
            elif k < 100: # over 100m2 per person, the additive value of m2 per person becomes almost obsolete
                ls.append('4 good') # below the 100m2 is still a good score
            else:
                ls.append('5 excellent') # above it will be excellent
        print(i[1].rsplit('.')[0], j)
        score_df = pd.concat([score_df, 
                              round(pd.Series(score, name = 'E2SCFA-Sc '+str(j)),2), 
                              pd.Series(ls, name = 'E2SCFA-class '+str(j))], 
                             axis = 1)
    E2SFCA_scores.append(score_df)
E2SFCA_scores[0]

preorocessing done

Abidjan 300
Abidjan 600
Abidjan 1000
Addis Ababa 300
Addis Ababa 600
Addis Ababa 1000
Baghdad 300
Baghdad 600
Baghdad 1000
Bangalore 300
Bangalore 600
Bangalore 1000
Belgrade 300
Belgrade 600
Belgrade 1000
Belo Horizonte 300
Belo Horizonte 600
Belo Horizonte 1000
Bogota 300
Bogota 600
Bogota 1000
Bologna 300
Bologna 600
Bologna 1000
Cali 300
Cali 600
Cali 1000
Canberra 300
Canberra 600
Canberra 1000
Casablanca 300
Casablanca 600
Casablanca 1000
Chandigarh 300
Chandigarh 600
Chandigarh 1000
Chelyabinsk 300
Chelyabinsk 600
Chelyabinsk 1000
Chengdu 300
Chengdu 600
Chengdu 1000
Christchurch 300
Christchurch 600
Christchurch 1000
Cochabamba 300
Cochabamba 600
Cochabamba 1000
Copenhagen 300
Copenhagen 600
Copenhagen 1000
Curitiba 300
Curitiba 600
Curitiba 1000
Damascus 300
Damascus 600
Damascus 1000
Detroit 300
Detroit 600
Detroit 1000
Dhaka 300
Dhaka 600
Dhaka 1000
Dongguan 300
Dongguan 600
Dongguan 1000
Fortaleza 300
Fortaleza 600
Fortaleza 1000
Ghent 300
Ghent 600
Ghen

,population,geometry,E2SCFA-Sc 300,E2SCFA-class 300,E2SCFA-Sc 600,E2SCFA-class 600,E2SCFA-Sc 1000,E2SCFA-class 1000
0,1,"POLYGON ((-4.00208 5.47625, -4.00125 5.47625, ...",0.0,0 no,0.0,0 no,0.00,0 no
1,1,"POLYGON ((-4.00125 5.47625, -4.00042 5.47625, ...",0.0,0 no,0.0,0 no,0.00,0 no
2,1,"POLYGON ((-4.00042 5.47625, -3.99958 5.47625, ...",0.0,0 no,0.0,0 no,0.00,0 no
3,32,"POLYGON ((-4.05042 5.46875, -4.04958 5.46875, ...",0.0,0 no,0.0,0 no,0.00,0 no
4,31,"POLYGON ((-4.04958 5.46875, -4.04875 5.46875, ...",0.0,0 no,0.0,0 no,0.00,0 no
...,...,...,...,...,...,...,...,...
62388,14,"POLYGON ((-4.09292 5.39458, -4.09208 5.39458, ...",0.0,0 no,0.0,0 no,35.08,4 good
62389,16,"POLYGON ((-4.09208 5.39458, -4.09125 5.39458, ...",0.0,0 no,0.0,0 no,0.00,0 no
62390,16,"POLYGON ((-4.09125 5.39458, -4.09042 5.39458, ...",0.0,0 no,0.0,0 no,0.00,0 no
62391,17,"POLYGON ((-4.09042 5.39458, -4.08958 5.39458, ...",0.0,0 no,0.0,0 no,0.00,0 no


In [59]:
%%time
scores = list()
write_path = 'D:/Dumps/GEE-WP Scores/city_summary/'
for i in range(len(ls_file)):
    score = pd.merge(E2SFCA_scores[i],cities_scores[i], on = 'geometry')
    
    score = score[['geometry','population',
                   'E2SCFA-class 300','E2SCFA-class 600','E2SCFA-class 1000',
                   'GravQnt 300','GravQnt 600','GravQnt 1000',
                   'E2SCFA-Sc 300','E2SCFA-Sc 600','E2SCFA-Sc 1000',
                   'Grav-Sc 300','Grav-Sc 600','Grav-Sc 1000']]
        
    # Adding categories without grid geoms attached to ensure that each category is in a plotted map
    ls2 = pd.DataFrame(list([
                [Polygon(),'5 excellent','5 excellent','5 excellent','5 excellent','5 excellent','5 excellent'],
                [Polygon(),'4 good','4 good','4 good','4 good','4 good','4 good'],
                [Polygon(),'3 sufficient','3 sufficient','3 sufficient','3 sufficient','3 sufficient','3 sufficient'],
                [Polygon(),'2 mediocre','2 mediocre','2 mediocre','2 mediocre','2 mediocre','2 mediocre'],
                [Polygon(),'1 low','1 low','1 low','1 low','1 low','1 low'],
                [Polygon(),'0 no','0 no','0 no','0 no','0 no','0 no']]))
    
    ls2.columns = ['geometry',
                   'E2SCFA-class 300','E2SCFA-class 600','E2SCFA-class 1000',
                   'GravQnt 300','GravQnt 600','GravQnt 1000']
    
    score = pd.concat([score, pd.DataFrame(ls2)])
    
    score = gpd.GeoDataFrame(score, geometry = 'geometry', crs = 4326)
    score.to_file(write_path+cities_adj['City'][i]+'.gpkg')
    score.iloc[:,score.columns != 'geometry'].to_csv(write_path+cities_adj['City'][i]+'.csv')
    print(cities_adj['City'][i])
    scores.append(score)

Abidjan
Addis Ababa
Baghdad
Bangalore
Belgrade
Belo Horizonte
Bogota
Bologna
Cali
Canberra
Casablanca
Chandigarh
Chelyabinsk
Chengdu
Christchurch
Cochabamba
Copenhagen
Curitiba
Damascus
Detroit
Dhaka
Dongguan
Fortaleza
Ghent
Guatemala City
Gwangju
Hanoi
Indore
Johannesburg
Kampala
Karlsruhe
Kathmandu
Kharkiv
Kuala Lumpur
Kumamoto
Lagos
Leicester
Liverpool
Lubumbashi
Lviv
Manila
Memphis
Mexico City
Mombasa
Nanjing
New Delhi
Oran
Oslo
Perth
Philadelphia
Pune
Rennes
Rosario
Rostov
San Antonio
San Diego
Santo Domingo
Seattle
Seoul
Sevilla
Shijiazhuang
Shiraz
Skopje
Surabaya
Tallinn
Tashkent
Tel Aviv
Toronto
Turin
Utrecht
Utsunomiya
Washington DC
Yaounde
Yerevan
Zibo
CPU times: total: 15min 19s
Wall time: 28min 59s


In [40]:
ls = []
ls2 = []
for i in enumerate(scores):
    ls = ls + (i[1]['E2SCFA-class 300'].value_counts().sort_index()-1).tolist()
    ls2 = ls2 + list(np.repeat(cities_adj['City'][i[0]],6))

In [44]:
import matplotlib.pyplot as plt

In [46]:
df = pd.DataFrame({'City':ls2,'E-300 class':ls})

In [64]:
summ_df = pd.DataFrame([i.groupby('E2SCFA-class 300')['population'].sum() / i['population'].sum() for i in scores])
summ_df.index = cities_adj['City']
summ_df
summ_df.to_csv(write_path+'E2SFCA_300_popweighaccess.csv')

In [65]:
summ_df = pd.DataFrame([i.groupby('GravQnt 300')['population'].sum() / i['population'].sum() for i in scores])
summ_df.index = cities_adj['City']
summ_df
summ_df.to_csv(write_path+'Gravity_300_popweighaccess.csv')

In [40]:
summ_df.sum(axis=1)

City
Bologna    0.0
Oslo       0.0
dtype: float64